<a href="https://colab.research.google.com/github/Cirediallo/Models/blob/main/icefall_asr_librispeech_pretrained_tdnn_lstm_ctc_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description
This notebook shows how to use a pre-trained tdnn-lstm_ctc model with [icefall](https://github.com/k2-fsa/icefall/).

## Environment setup

To use a pre-trained model with icefall, we have to install the following dependencies:

- [k2][k2], for FSA operations
- [torchaudio][audio], for reading sound files
- [kaldifeat][kaldifeat], for extracting features from a single sound
  file or multiple sound files

**NOTE**: [lhotse][lhotse] is used only in training time, for data preparation.


[k2]: https://github.com/k2-fsa/k2
[audio]: https://github.com/pytorch/audio
[kaldifeat]: https://github.com/csukuangfj/kaldifeat
[lhotse]: https://github.com/lhotse-speech/lhotse

### Install PyTorch and torchaudio

In [ ]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


Colab provides CUDA 11.0, so we choose to install torch==1.7.1+cu110.

The reason to select torch 1.7.1 is that we are going to install k2 using `pip install`, which depends on torch 1.7.1.

If you want to use a different version of PyTorch, please
refer to <https://k2.readthedocs.io/en/latest/installation/index.html> for installing k2 either from source or with `conda install`.

In [2]:
#! pip install torch==1.7.1+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

### Install k2

In [ ]:
#! pip install k2==1.17

In [ ]:
#! pip install k2==1.17.dev20220725+cuda11.0.torch1.7.1 -f https://k2-fsa.org/nightly/index.html

Check that k2 was installed successfully:

In [ ]:
#! python3 -m k2.version

## All previous steps in one

### Install Cuda and cudnn

In [7]:
! wget https://developer.download.nvidia.com/compute/cuda/11.7.1/local_installers/cuda_11.7.1_515.65.01_linux.run

--2023-05-01 15:18:38--  https://developer.download.nvidia.com/compute/cuda/11.7.1/local_installers/cuda_11.7.1_515.65.01_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3524358811 (3.3G) [application/octet-stream]
Saving to: ‘cuda_11.7.1_515.65.01_linux.run’

cuda_11.7.1_515.65. 100%[===================>]   3.28G   248MB/s    in 13s     

2023-05-01 15:18:51 (258 MB/s) - ‘cuda_11.7.1_515.65.01_linux.run’ saved [3524358811/3524358811]



In [8]:
! chmod +x /content/cuda_11.7.1_515.65.01_linux.run

In [9]:
! cd /content/ && \
  ./cuda_11.7.1_515.65.01_linux.run \
    --silent \
    --toolkit \
    --installpath=/content/cuda11/software/cuda-11.7.1 \
    --no-opengl-libs \
    --no-drm \
    --no-man-page

In [10]:
! wget https://huggingface.co/csukuangfj/cudnn/resolve/main/cudnn-11.3-linux-x64-v8.2.0.53.tgz

--2023-05-01 15:21:02--  https://huggingface.co/csukuangfj/cudnn/resolve/main/cudnn-11.3-linux-x64-v8.2.0.53.tgz
Resolving huggingface.co (huggingface.co)... 18.160.249.9, 18.160.249.78, 18.160.249.70, ...
Connecting to huggingface.co (huggingface.co)|18.160.249.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/csukuangfj/cudnn/7a195dc93a7cda2bdd4d9b73958d259c784be422cd941a9a625aab75309f19dc?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27cudnn-11.3-linux-x64-v8.2.0.53.tgz%3B+filename%3D%22cudnn-11.3-linux-x64-v8.2.0.53.tgz%22%3B&Expires=1683213662&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL2NzdWt1YW5nZmovY3Vkbm4vN2ExOTVkYzkzYTdjZGEyYmRkNGQ5YjczOTU4ZDI1OWM3ODRiZTQyMmNkOTQxYTlhNjI1YWFiNzUzMDlmMTlkYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE2ODMyMTM2NjJ9fX1dfQ__&Signature=gq1zaVdfAXzgssk00K6kglFRmsMBcFlqa8F

In [11]:
! cd /content/ && tar xvf ./cudnn-11.3-linux-x64-v8.2.0.53.tgz --strip-components=1 -C /content/cuda11/software/cuda-11.7.1

cuda/include/cudnn.h
cuda/include/cudnn_adv_infer.h
cuda/include/cudnn_adv_infer_v8.h
cuda/include/cudnn_adv_train.h
cuda/include/cudnn_adv_train_v8.h
cuda/include/cudnn_backend.h
cuda/include/cudnn_backend_v8.h
cuda/include/cudnn_cnn_infer.h
cuda/include/cudnn_cnn_infer_v8.h
cuda/include/cudnn_cnn_train.h
cuda/include/cudnn_cnn_train_v8.h
cuda/include/cudnn_ops_infer.h
cuda/include/cudnn_ops_infer_v8.h
cuda/include/cudnn_ops_train.h
cuda/include/cudnn_ops_train_v8.h
cuda/include/cudnn_v8.h
cuda/include/cudnn_version.h
cuda/include/cudnn_version_v8.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.8
cuda/lib64/libcudnn.so.8.2.0
cuda/lib64/libcudnn_adv_infer.so
cuda/lib64/libcudnn_adv_infer.so.8
cuda/lib64/libcudnn_adv_infer.so.8.2.0
cuda/lib64/libcudnn_adv_train.so
cuda/lib64/libcudnn_adv_train.so.8
cuda/lib64/libcudnn_adv_train.so.8.2.0
cuda/lib64/libcudnn_cnn_infer.so
cuda/lib64/libcudnn_cnn_infer.so.8
cuda/lib64/libcudnn_cnn_infer.so.8.2.0
cuda/lib64/

In [13]:
import os
os.environ['CUDA_HOME'] = "/content/cuda11/software/cuda-11.7.1"
os.environ['PATH'] += ":/content/cuda11/software/cuda-11.7.1/bin"
os.environ['LD_LIBRARY_PATH'] += ":/content/cuda11/software/cuda-11.7.1/lib64"
os.environ['LD_LIBRARY_PATH'] += ":/content/cuda11/software/cuda-11.7.1/lib"

os.environ['CUDA_TOOLKIT_ROOT_DIR']=os.environ['CUDA_HOME']
os.environ['CUDA_TOOLKIT_ROOT']=os.environ['CUDA_HOME']
os.environ['CUDA_BIN_PATH']=os.environ['CUDA_HOME']
os.environ['CUDA_PATH']=os.environ['CUDA_HOME']
os.environ['CUDA_INC_PATH']="/content/cuda11/software/cuda-11.7.1/targets/x86_64-linux"


In [3]:
!pip install torch==1.13.1
!pip install torchaudio==0.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch

In [4]:
! git clone https://github.com/k2-fsa/k2.git

Cloning into 'k2'...
remote: Enumerating objects: 14302, done.
remote: Counting objects: 100% (574/574), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 14302 (delta 440), reused 558 (delta 433), pack-reused 13728
Receiving objects: 100% (14302/14302), 15.34 MiB | 26.90 MiB/s, done.
Resolving deltas: 100% (10010/10010), done.


In [5]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [14]:
!cd /content/k2 && \
  export K2_MAKE_ARGS="-j6" && \
  python3 setup.py install


running install
/usr/lib/python3.10/distutils/cmd.py:62: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/lib/python3.10/distutils/cmd.py:62: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917

In [2]:
#pip install k2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 MB 17.3 MB/s eta 0:00:00


In [15]:
! python3 -m k2.version


k2 version: 1.24.1
Build type: Release
Git SHA1: c51a0b9684442a88ee37f3ce0af686a04b66855b
Git date: Mon May 1 13:38:03 2023
Cuda used to build k2: 11.7
cuDNN used to build k2: 8.2.0
Python version used to build k2: 3.10
OS used to build k2: Ubuntu 20.04.5 LTS
CMake version: 3.25.2
GCC version: 9.4.0
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -use_fast_math -

### Install kaldifeat

In [16]:
! pip install kaldifeat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 482.4/482.4 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaldifeat: filename=kaldifeat-1.21-cp310-cp310-linux_x86_64.whl size=263257 sha256=89a29d27b94c63f1c78097de662bcc2e3d1c983259b7b331c24cf3d5f28f7ce6
  Stored in directory: /root/.cache/pip/wheels/e3/31/9b/a97ec285f4e5bfb43d7f85cff15db32ed680d3619e15675643
Successfully built kaldifeat


To check that kaldifeat was installed successfully, run

In [17]:
! python3 -c "import kaldifeat; print(kaldifeat.__version__)"

1.21


### Install icefall

icefall is a collection of Python scripts. All you need is just to
download its source code and set the `PYTHONPATH` environment variable.

In [18]:
! git clone https://github.com/k2-fsa/icefall

Cloning into 'icefall'...
remote: Enumerating objects: 11840, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 11840 (delta 89), reused 96 (delta 45), pack-reused 11657
Receiving objects: 100% (11840/11840), 13.70 MiB | 32.79 MiB/s, done.
Resolving deltas: 100% (8037/8037), done.


In [19]:
! pip install -q kaldialign

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.7 MB/s eta 0:00:00


### install lhoste

In [20]:
! pip install git+https://github.com/lhotse-speech/lhotse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-l9zb7xgw
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-l9zb7xgw
  Resolved https://github.com/lhotse-speech/lhotse to commit cbd7b2b3e57b0e97470fe015eca5eafa6bb85d5e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 10.6 MB/s eta 0:00:00
  Created wheel for lhotse: filename=lhotse-1.15.0.dev0+git.cbd7b2b.clean-py3-none-any.whl size=639690 sha256=cb57064e9642c7cd729913662b004ba2f7b0341b362a6c9384dd79ed8f81a2e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-hldx0vyb/wheels/df/b0/ff/cce0f1686

In [21]:
! cd icefall && \
  ! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00


In [22]:
import os

!pwd
! echo $PYTHONPATH


os.environ['PYTHONPATH'] += ":/content/icefall"

! echo $PYTHONPATH

/content
/env/python
/env/python:/content/icefall


In [ ]:
#export PYTHONPATH=/content:$PYTHONPATH

## Download pre-trained Tdnn-Lstm CTC model

To make the following steps easier, we decide to download the model
to `icefall/egs/librispeech/ASR/tmp-lstm`

In [ ]:
! apt-get install -y -qq tree sox git-lfs

Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.5-1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.5-1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack .../1-libopencore-amrwb0_0.1.5-1_amd64.deb ...
Unpacking libopencore-amrwb0:amd64 (0.1.5-1) ...
Selecting previously unselected package libsox3:amd64.
Preparing to unpack .../2-libsox3_14.4.2+git20190427-2+deb11u2build0.20.04.1_amd64.deb ...
Unpacking libsox3:amd64 (14.4.2+git20190427-2+deb11u2build0.20.04.1) ...
Selecting previously unselected package libsox-fmt-alsa:amd64.
Preparing to unpack .../3-libsox-fmt-alsa_14.4.2+git20190427-2+deb11u2build0.20.04.1_amd64.deb ...
Unpacking libsox-fmt-alsa:amd64 (14.4.2+git20190427-2+deb11u2build0.20.04.1) ...
Selecting previously unselected package libsox-fmt-base:amd64.
Preparing to unpack .../4-libsox-fmt-b

**CAUTION**: You have to run `sudo apt-get install git-lfs`
and `git lfs install` in order to download the pre-trained model.

In [ ]:
! cd icefall/egs/librispeech/ASR && rm -rf tmp-lstm

In [ ]:
! cd icefall/egs/librispeech/ASR && \
  mkdir tmp-lstm && \
  cd tmp-lstm && \
  git lfs install && \
  git clone https://huggingface.co/pkufool/icefall_asr_librispeech_tdnn-lstm_ctc && \
  cd icefall_asr_librispeech_tdnn-lstm_ctc && \
  cd ../.. && \
  tree tmp-lstm

In [ ]:
! ffprobe -show_format icefall/egs/librispeech/ASR/tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1089-134686-0001.flac

In [ ]:
! ffprobe -show_format icefall/egs/librispeech/ASR/tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0001.flac

In [ ]:
! ffprobe -show_format icefall/egs/librispeech/ASR/tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0002.flac

Due to limited memory provided by Colab, you have to upgrade to Colab Pro
to run `HLG decoding + LM rescoring`.

In [ ]:
! pip install lhotse sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## HLG decoding (1best)

In [ ]:
! cd icefall/egs/librispeech/ASR && \
    PYTHONPATH=/content/icefall python3 ./tdnn_lstm_ctc/pretrained.py \
      --method 1best \
      --checkpoint ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt \
      --words-file ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt \
      --HLG ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1089-134686-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0002.flac 

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
2022-11-02 14:32:26,910 INFO [pretrained.py:162] {'feature_dim': 80, 'subsampling_factor': 3, 'num_classes': 72, 'sample_rate': 16000, 'search_beam': 20, 'output_beam': 5, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt', 'words_file': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt', 'HLG': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt', 'method': '1best', 'G': None, 'ngram_lm_scale': 0.8, 'sound_files': ['./tmp-lstm/icefall_asr_librispeech_tdnn-

## HLG decoding + LM rescoring

In [ ]:
! cd icefall/egs/librispeech/ASR && \
    PYTHONPATH=/content/icefall python3 ./tdnn_lstm_ctc/pretrained.py \
      --method whole-lattice-rescoring \
      --checkpoint ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt \
      --words-file ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt \
      --HLG ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt \
      --G ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lm/G_4_gram.pt \
      --ngram-lm-scale 0.08 \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1089-134686-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0002.flac 

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
2022-11-02 14:35:30,881 INFO [pretrained.py:162] {'feature_dim': 80, 'subsampling_factor': 3, 'num_classes': 72, 'sample_rate': 16000, 'search_beam': 20, 'output_beam': 5, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt', 'words_file': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt', 'HLG': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt', 'method': 'whole-lattice-rescoring', 'G': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lm/G_4_gram.p

## Training

### Data preparation

In [1]:
! cd icefall/egs/librispeech/ASR && \
  ./prepare.sh

2023-05-01 16:26:54 (prepare.sh:63:main) dl_dir: /content/icefall/egs/librispeech/ASR/download
2023-05-01 16:26:54 (prepare.sh:66:main) Stage -1: Download LM
Traceback (most recent call last):
  File "/content/icefall/egs/librispeech/ASR/./local/download_lm.py", line 44, in <module>
    from lhotse.utils import urlretrieve_progress
ImportError: cannot import name 'urlretrieve_progress' from 'lhotse.utils' (/usr/local/lib/python3.10/dist-packages/lhotse/utils.py)


### Launch training

In [ ]:
# https://k2-fsa.github.io/icefall/recipes/Non-streaming-ASR/librispeech/tdnn_lstm_ctc.html